In [1]:
import os
import torch
import time
import numpy as np
import torch.nn as nn
from tqdm import tqdm
from torch.cuda import amp
from scipy.io import savemat
import torch.nn.functional as F
from torchvision import datasets, transforms

from Network.ReActNet_18_Qa import *
from Network.ReActNet_A_Qa import *
from Network.utils import *

os.environ["CUDA_VISIBLE_DEVICES"] = '0'

In [2]:
Begin_epoch = 0
Max_epoch = 128
Learning_rate = 1e-3
Weight_decay = 5e-6
Momentum = 0.9
Top_k = 5
AMP = False

Dataset_path = 'tests/data/CIFAR10/'
Batch_size = 256
Workers = 8
Targetnum = 10

Test_every_iteration = None
Name_suffix = '_step1'
Savemodel_path = './savemodels/'
Record_path = './recorddata/'
if not os.path.exists(Savemodel_path):
    os.mkdir(Savemodel_path)
if not os.path.exists(Record_path):
    os.mkdir(Record_path)

In [3]:
_seed_ = 2023
torch.manual_seed(_seed_)
np.random.seed(_seed_)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

In [4]:
transform_train = transforms.Compose([
    transforms.Pad(4, padding_mode='reflect'),
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])
 
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])

Train_data = datasets.CIFAR10(root=Dataset_path, train=True, download=True, transform=transform_train)
Test_data = datasets.CIFAR10(root=Dataset_path, train=False, download=True, transform=transform_test)

train_data_loader = torch.utils.data.DataLoader(
    dataset=Train_data,
    batch_size=Batch_size,
    shuffle=True,
    num_workers=Workers, 
    pin_memory=True,
    drop_last=True
)

test_data_loader = torch.utils.data.DataLoader(
    dataset=Test_data,
    batch_size=Batch_size,
    shuffle=False,
    num_workers=Workers, 
    pin_memory=True,
    drop_last=False
)

In [5]:
# net = ResNet18(num_classes=Targetnum, imagenet=False)
net = Reactnet(num_classes=Targetnum, imagenet=False)

In [7]:
#net = nn.DataParallel(net).cuda()
net = net.cuda()
max_test_acc = 0.
if Begin_epoch!=0:
    net.load_state_dict(torch.load(Savemodel_path + f'epoch{Begin_epoch-1}{Name_suffix}.h5'))
    max_test_acc = np.load(Savemodel_path + f'max_acc{Name_suffix}.npy')
    max_test_acc = max_test_acc.item()

scaler = amp.GradScaler() if AMP else None
Test_top1 = []
Test_topk = []
Test_lossall = []
Epoch_list = []
Iteration_list = []

In [8]:
all_parameters = net.parameters()
weight_parameters = []
for pname, p in net.named_parameters():
    if (p.ndimension() == 4 or 'conv' in pname) and 'L_alpha' not in pname:
        weight_parameters.append(p)
weight_parameters_id = list(map(id, weight_parameters))
other_parameters = list(filter(lambda p: id(p) not in weight_parameters_id, all_parameters))

criterion_train = DistributionLoss()
criterion_test = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam([
    {'params' : other_parameters, 'weight_decay' : 0., 'initial_lr': Learning_rate},
    {'params' : weight_parameters, 'weight_decay' : Weight_decay, 'initial_lr': Learning_rate}],
    lr = Learning_rate)
lr_scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lambda step : (1.0-step/Max_epoch), last_epoch=Begin_epoch-1)

In [9]:
def test_model(net, max_test_acc, data_loader=test_data_loader, criterion=criterion_test, epoch=None, iteration=None, record=True):
    net.eval()
    test_samples = 0
    test_loss = 0
    test_acc_top1 = 0
    test_acc_topk = 0
    
    with torch.no_grad():
        for img, label in tqdm(data_loader):
            img = img.cuda()
            label = label.cuda()
            label_onehot = F.one_hot(label, Targetnum).float()
            
            out_fr = net(img)
            loss = criterion(out_fr, label)
                
            test_samples += label.numel()
            test_loss += loss.item() * label.numel()

            test_acc_top1 += (out_fr.argmax(1) == label).float().sum().item()
            _, pred = out_fr.topk(Top_k, 1, True, True)
            test_acc_topk += torch.eq(pred, label.view(-1,1)).float().sum().item()
    
    test_loss /= test_samples
    test_acc_top1 /= test_samples
    test_acc_topk /= test_samples

    if test_acc_top1 >= max_test_acc:
        max_test_acc = test_acc_top1
        torch.save(net.state_dict(), Savemodel_path + f'max_acc{Name_suffix}.h5')
        np.save(Savemodel_path + f'max_acc{Name_suffix}.npy', np.array(max_test_acc))

    if record:
        assert epoch is not None, "epoch is None!"
        assert iteration is not None, "iteration is None!"
        
        Epoch_list.append(epoch+1)
        Iteration_list.append(iteration+1)
        Test_top1.append(test_acc_top1)
        Test_topk.append(test_acc_topk)
        Test_lossall.append(test_loss)

        record_data = np.array([Epoch_list, Iteration_list, Test_top1, Test_topk, Test_lossall]).T
        mdic = {f'Record_data':record_data, f'Record_meaning':['Epoch_list', 'Iteration_list', 'Test_top1', f'Test_top{Top_k}', 'Test_loss']}

        savemat(Record_path + f'Test_{Begin_epoch}_{epoch}{Name_suffix}.mat',mdic)
        if os.path.exists(Record_path + f'Test_{Begin_epoch}_{epoch-1}{Name_suffix}.mat'):
            os.remove(Record_path + f'Test_{Begin_epoch}_{epoch-1}{Name_suffix}.mat')

    return test_loss, test_acc_top1, test_acc_topk, max_test_acc

In [10]:
def train_model(net, max_test_acc, epoch, data_loader=train_data_loader, optimizer=optimizer, criterion=criterion_test, scaler=scaler, record=True):
    train_samples = 0
    train_loss = 0
    train_acc_top1 = 0
    train_acc_topk = 0
    
    for i, (img, label) in enumerate(tqdm(data_loader)):
        net.train()
        img = img.cuda()
        label = label.cuda()
        label_onehot = F.one_hot(label, Targetnum).float()
        
        if AMP:
            with amp.autocast():
                out_fr = net(img)
                loss = criterion(out_fr, label)
        else:
            out_fr = net(img)
            loss = criterion(out_fr, label)
            
        train_samples += label.numel()
        train_loss += loss.item() * label.numel()

        train_acc_top1 += (out_fr.argmax(1) == label).float().sum().item()
        _, pred = out_fr.topk(Top_k, 1, True, True)
        train_acc_topk += torch.eq(pred, label.view(-1,1)).float().sum().item()
        
        optimizer.zero_grad()
        if AMP:
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
        else:
            loss.backward()

            parameters_list = []
            for name, p in net.named_parameters():
                if not 'fc' in name:
                    parameters_list.append(p)
            adaptive_clip_grad(parameters_list, clip_factor=0.001)
            
            optimizer.step()

        if Test_every_iteration is not None:
            if (i+1) % Test_every_iteration == 0:
                test_loss, test_acc_top1, test_acc_topk, max_test_acc = test_model(net, max_test_acc, epoch=epoch, iteration=i, record=record)
                print(f'Test_loss: {test_loss:.4f}, Test_acc_top1: {test_acc_top1:.4f}, Test_acc_top{Top_k}: {test_acc_topk:.4f}, Max_test_acc: {max_test_acc:.4f}')
    
    train_loss /= train_samples
    train_acc_top1 /= train_samples
    train_acc_topk /= train_samples

    test_loss, test_acc_top1, test_acc_topk, max_test_acc = test_model(net, max_test_acc, epoch=epoch, iteration=i, record=record)
        
    return train_loss, train_acc_top1, train_acc_topk, test_loss, test_acc_top1, test_acc_topk, max_test_acc

In [11]:
for epoch in range(Begin_epoch, Max_epoch):

    start_time = time.time()
    train_loss, train_acc_top1, train_acc_topk, test_loss, test_acc_top1, test_acc_topk, max_test_acc = train_model(net, max_test_acc, epoch)
    
    for param_group in optimizer.param_groups:
        lr = param_group['lr']
        
    lr_scheduler.step()

    print(f'''epoch={epoch}, train_acc_top1={train_acc_top1:.4f}, train_acc_top{Top_k}={train_acc_topk:.4f}, train_loss={train_loss:.4f}, test_top1={test_acc_top1:.4f}, test_top{Top_k}={test_acc_topk:.4f}, test_loss={test_loss:.4f}, max_test_acc={max_test_acc:.4f}, total_time={(time.time() - start_time):.4f}, LR={lr:.8f}''')
    
    torch.save(net.state_dict(), Savemodel_path + f'epoch{epoch}{Name_suffix}.h5')
    if os.path.exists(Savemodel_path + f'epoch{epoch-1}{Name_suffix}.h5'):
        os.remove(Savemodel_path + f'epoch{epoch-1}{Name_suffix}.h5')

100%|██████████| 40/40 [00:03<00:00, 12.43it/s]


epoch=0, train_acc_top1=0.3456, train_acc_top5=0.8446, train_loss=1.8492, test_top1=0.4628, test_top5=0.9337, test_loss=1.4323, max_test_acc=0.4628, total_time=60.5336, LR=0.00100000


100%|██████████| 40/40 [00:03<00:00, 12.42it/s]


epoch=1, train_acc_top1=0.5087, train_acc_top5=0.9362, train_loss=1.3508, test_top1=0.5602, test_top5=0.9540, test_loss=1.2087, max_test_acc=0.5602, total_time=59.5371, LR=0.00099219


100%|██████████| 40/40 [00:03<00:00, 12.48it/s]


...

epoch=125, train_acc_top1=0.9960, train_acc_top5=1.0000, train_loss=0.0165, test_top1=0.8797, test_top5=0.9955, test_loss=0.5151, max_test_acc=0.8798, total_time=53.5106, LR=0.00002344


100%|██████████| 40/40 [00:02<00:00, 13.85it/s]


epoch=126, train_acc_top1=0.9960, train_acc_top5=1.0000, train_loss=0.0162, test_top1=0.8788, test_top5=0.9952, test_loss=0.5141, max_test_acc=0.8798, total_time=53.4590, LR=0.00001563


100%|██████████| 40/40 [00:02<00:00, 13.76it/s]


epoch=127, train_acc_top1=0.9960, train_acc_top5=1.0000, train_loss=0.0158, test_top1=0.8801, test_top5=0.9953, test_loss=0.5118, max_test_acc=0.8801, total_time=53.6622, LR=0.00000781


In [12]:
net.load_state_dict(torch.load(Savemodel_path + f'max_acc{Name_suffix}.h5'))

<All keys matched successfully>

In [13]:
Confusion_Matrix = torch.zeros((Targetnum, Targetnum))
net.eval()
with torch.no_grad():
    for img, label in tqdm(test_data_loader):
        img = img.cuda()
        label = label.cuda()
        out_fr = net(img)
        guess = out_fr.argmax(1)
        for j in range(len(label)):
            Confusion_Matrix[label[j],guess[j]] += 1
acc = Confusion_Matrix.diag()
acc = acc.sum()/Confusion_Matrix.sum()
print(f'Confusion_Matrix = {Confusion_Matrix}')
print(f'acc = {acc}')

100%|██████████| 40/40 [00:03<00:00, 10.61it/s]

Confusion_Matrix = tensor([[907.,   8.,  15.,   5.,   6.,   3.,   4.,   4.,  29.,  19.],
        [  9., 944.,   4.,   1.,   0.,   0.,   1.,   1.,   9.,  31.],
        [ 32.,   1., 828.,  31.,  35.,  16.,  40.,  11.,   4.,   2.],
        [ 16.,   2.,  40., 759.,  23., 107.,  25.,  11.,   8.,   9.],
        [ 11.,   2.,  41.,  33., 844.,  16.,  27.,  23.,   1.,   2.],
        [  5.,   0.,  17., 104.,  29., 820.,  12.,  10.,   1.,   2.],
        [  4.,   2.,  23.,  24.,  12.,   6., 923.,   3.,   1.,   2.],
        [ 12.,   1.,  12.,  20.,  20.,  25.,   5., 904.,   0.,   1.],
        [ 30.,   7.,   1.,   5.,   2.,   1.,   1.,   1., 940.,  12.],
        [ 12.,  35.,   1.,   3.,   0.,   0.,   5.,   2.,  10., 932.]])
acc = 0.8801000118255615


In [14]:
net

DataParallel(
  (module): Reactnet(
    (feature): ModuleList(
      (0): firstconv3x3(
        (conv1): ScaledStdConv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (1): BasicBlock(
        (move11): LearnableBias()
        (binary_3x3): ScaledStdConv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (move12): LearnableBias()
        (prelu1): Q_PReLU()
        (move13): LearnableBias()
        (move21): LearnableBias()
        (binary_pw_down1): ScaledStdConv2d(32, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (binary_pw_down2): ScaledStdConv2d(32, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (move22): LearnableBias()
        (prelu2): Q_PReLU()
        (move23): LearnableBias()
        (binary_activation): BinaryActivation()
        (pooling): AvgPool2d(kernel_size=2, stride=2, padding=0)
      )
      (2): BasicBlock(
        (move11): LearnableBias()
        (binary_3x3): ScaledStdConv